In [1]:
%pip install --upgrade --quiet pip langchain sentence_transformers langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",  #input text longer than 384 word pieces is truncated.
    model_kwargs={
        'device': 'cpu'
    },
    show_progress=True
)

/Users/madhurjajoo/MyData/Coding/genai-notebooks/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/madhurjajoo/MyData/Coding/genai-notebooks/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("./docs/state_of_the_union.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=256, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

db = Chroma.from_documents(documents=docs, embedding=embeddings)

Created a chunk of size 304, which is longer than the specified 256
Created a chunk of size 332, which is longer than the specified 256
Created a chunk of size 281, which is longer than the specified 256
Created a chunk of size 325, which is longer than the specified 256
Batches: 100%|██████████| 7/7 [00:03<00:00,  1.96it/s]


In [17]:
query = "Why was NATO alliance created?"
docs = db.similarity_search_with_score(query)

print(docs[0])

Batches: 100%|██████████| 1/1 [00:00<00:00, 13.80it/s]

(Document(page_content='For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia.', metadata={'source': './docs/state_of_the_union.txt'}), 0.8522162437438965)


In [18]:
retriever = db.as_retriever(search_type="mmr")
retriever.invoke(query)[0]

Batches: 100%|██████████| 1/1 [00:00<00:00,  8.00it/s]


Document(page_content='For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia.', metadata={'source': './docs/state_of_the_union.txt'})